In [11]:
import numpy as np
import pandas as pd

data = pd.read_csv('./data/capture20110811.pcap.netflow.labeled', sep='\s+')

data[['Src_IP_Addr', 'Src_Port']] = data['Src_IP_Addr:Port'].str.split(':', n=1, expand=True)
data[['Dst_IP_Addr', 'Dst_Port']] = data['Dst_IP_Addr:Port'].str.split(':', n=1, expand=True)

data['Label'] = data['Label(LEGITIMATE:Botnet:Background)']

data = data.drop(labels=['->', 'Label(LEGITIMATE:Botnet:Background)', 'Src_IP_Addr:Port', 'Dst_IP_Addr:Port'], axis=1)

print(data.shape)
print(data.columns)

data.head()

(6351188, 14)
Index(['#Date_flow', 'start', 'Durat', 'Prot', 'Flags', 'Tos', 'Packets',
       'Bytes', 'Flows', 'Src_IP_Addr', 'Src_Port', 'Dst_IP_Addr', 'Dst_Port',
       'Label'],
      dtype='object')


,#Date_flow,start,Durat,Prot,Flags,Tos,Packets,Bytes,Flows,Src_IP_Addr,Src_Port,Dst_IP_Addr,Dst_Port,Label
0,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,135,1,89.31.8.11,23929,147.32.84.229,13363,Background
1,2011-08-11,10:10:00.003,0.967,UDP,INT,0,2,276,1,147.32.84.229,13363,89.31.8.11,23929,Background
2,2011-08-11,10:10:00.006,0.000,UDP,INT,0,1,62,1,208.88.186.6,34042,147.32.84.229,13363,Background
3,2011-08-11,10:10:00.008,0.000,UDP,INT,0,1,78,1,92.118.218.77,55246,147.32.84.229,13363,Background
4,2011-08-11,10:10:00.009,0.000,UDP,INT,0,1,72,1,182.185.139.181,10223,147.32.84.229,13363,Background


In [9]:
infected_IP = '147.32.84.165:*'